In [57]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_ollama.llms import OllamaLLM
from langchain.chains.summarize import load_summarize_chain

In [58]:
#OPENAI_API_KEY = '...'

In [59]:
%pip install --upgrade --quiet  youtube-transcript-api
%pip install --upgrade --quiet  pytube

3991.89s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.
3997.86s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


In [62]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=pNcQ5XXMgH4&list=PLqZXAkvF1bPNQER9mLmDbntNfSpzdDIU5&index=9", add_video_info=False
)

In [63]:
loader.load()

[Document(metadata={'source': 'pNcQ5XXMgH4'}, page_content="what is going on good people again right now we have a super exciting tutorial because we are going to take YouTube transcripts and we're going to pass them to open Ai and the way that we're going to do that is via a library called Lang chain which is what this entire series is about now before we jumped into it I wanted to show a diagram again I think these diagrams are helpful but you have to let me know so just let me know in the comments here so I wanted to do an overview about what we're actually going to be writing out in code because I think it's a little easier to see in pictures first so the way this is going to work is we're going to have a video a YouTube video we're going to pass it we're going to pass it a URL and then what Lang chain is going to help us do is it's going to help us load this video as a document and a document just means you're going to be taking the transcript which is the text of the video and yo

### 1. Simple Videos

In [69]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=pNcQ5XXMgH4&list=PLqZXAkvF1bPNQER9mLmDbntNfSpzdDIU5&index=9", add_video_info=False)

In [70]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=pNcQ5XXMgH4&list=PLqZXAkvF1bPNQER9mLmDbntNfSpzdDIU5&index=9", add_video_info=False
)

In [71]:
result = loader.load()

In [72]:
print (type(result))
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print ("")
print (result)

<class 'list'>


KeyError: 'author'

In [74]:
#llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [9]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

' Pedro Pascal shared his experience shooting for HBO\'s "The Last of Us" and his journey to becoming a recognizable actor. He also spoke about his Chilean family, who were initially not proud of his acting career, but are now supportive. He concluded his speech by introducing Coldplay, who will be performing at the show.'

In [75]:
llm = OllamaLLM(model="llama3.2-vision", temperature=0.9)

### 2. Long Videos

In [10]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=6Ub7Z1AGIuk", add_video_info=True)

In [11]:
result = loader.load()

In [12]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 14199 tokens (13943 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

Problem, your transcript/document is too long

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [26]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [27]:
texts = text_splitter.split_documents(result)

In [31]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts[:4])



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"somehow five years later covet hits and you know what they decide to do they're like you know what let's go all in on creating like safety safety stuff for covert like masks respirators um gloves things like that and they become one of the like core providers right at the start of covet they shifted all their production manufacturing stuff they're like forget the vodka bottles forget the LED screens we need n95 masks and we need respirators and we need you know these testing kits and we need and they started producing this stuff and they started just winning all these contracts [Music] we're live Sean uh we have a bunch of stuff you you have a you have a a full menu over here and so do I yeah all right I want you to look at my menu here and I want you to just pick something off the menu go ahead and I'll Riff Off it we don't prefer people who they don't they don't kno



> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"

Covet shifted their production from vodka bottles and LED screens to safety equipment such as masks, respirators, and gloves when the pandemic hit. They won contracts and became one of the core providers. They also created a snow machine to create a snow hill in their driveway for sledding without going to Tahoe.

 A company created in 2014 developed a vodka bottle with an LED sign that could be programmed with any message. When the COVID-19 pandemic hit, they shifted their production to safety equipment such as masks, respirators, gloves, and testing kits. They have since become a successful provider of safety equipment, winning contracts with the government and becoming the sole testing provider for the Lakers Stadium.


Levi's Stadium and other places are offering contracts worth over $100 million for products such as masks. During the COVID-19 pandemic, people were able to tak

'\n\nCovet shifted their production from vodka bottles and LED screens to safety equipment such as masks, respirators, and gloves when the pandemic hit, winning contracts and becoming one of the core providers. They also created a snow machine to create a snow hill in their driveway for sledding. During the pandemic, people were able to take advantage of the opportunity to make a lot of money, with one friend making $2 million a month and another going public with 85 million in revenue. The speaker encouraged their father to take a risk and take action, which he learned from a Tony Robbins event.'

### 3. Multiple Videos

In [32]:
youtube_url_list = ["https://www.youtube.com/watch?v=AXq0QHUwmh8", "https://www.youtube.com/watch?v=EwHrjZxAT7g"]

texts = []

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

for url in youtube_url_list:
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    result = loader.load()
    
    texts.extend(text_splitter.split_documents(result))

In [33]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)

' This article discusses the process of building a backyard golf green, including the cost of materials and the steps involved. Bella Ramsey and Pedro Pascal also discuss their experiences on the set of Game of Thrones and The Last of Us, including their first impressions of each other, what they took from set, and what item from the Game of Thrones world they would bring to The Last of Us. Bella talks about her most prized possession, a tear stick, and her best attribute in a survival scenario. Finally, they discuss their first jobs.'